In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from random import shuffle

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
input_tensor   = tf.placeholder(shape=(None, 28, 28, 1), dtype="float32", name="input_tensor")
output_tensor  = tf.placeholder(shape=(None, 10), dtype="float32", name="output_tensor")
bn_tensor      = tf.placeholder(tf.bool,shape=None)
dropout_tensor = tf.placeholder(shape=(None), dtype="float32", name = "Dropout")

In [4]:
layer_1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2, 2))(input_tensor)
layer_2 = tf.keras.layers.BatchNormalization()(layer_1, training = bn_tensor)
layer_3 = tf.keras.layers.ReLU()(layer_1)
layer_3 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_3)

layer_4 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2))(layer_3)
layer_5 = tf.keras.layers.BatchNormalization()(layer_4, training = bn_tensor)
layer_6 = tf.keras.layers.ReLU()(layer_4)
layer_6 = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_6)


layer_7  = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (2, 2))(layer_6)
layer_8  = tf.keras.layers.BatchNormalization()(layer_7,training = bn_tensor)
layer_9  = tf.keras.layers.ReLU()(layer_7)
layer_9  = tf.keras.layers.Dropout(rate = dropout_tensor)(layer_9)



layer_10 = tf.keras.layers.AveragePooling2D(pool_size= 2, strides =1)(layer_9) 


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
layer_11 = tf.keras.layers.Dense(units= 10)(layer_10)
layer_12 = tf.reshape(layer_11,shape = (-1, 10))

In [6]:
print(layer_1)
print(layer_2)
print(layer_3)
print(layer_4)
print(layer_5)
print(layer_6)
print(layer_7)
print(layer_8)
print(layer_9)
print(layer_10)
print(layer_11)
print(layer_12)


Tensor("conv2d/BiasAdd:0", shape=(?, 13, 13, 32), dtype=float32)
Tensor("batch_normalization/cond/Merge:0", shape=(?, 13, 13, 32), dtype=float32)
Tensor("dropout_1/cond/Merge:0", shape=(?, 13, 13, 32), dtype=float32)
Tensor("conv2d_1/BiasAdd:0", shape=(?, 6, 6, 64), dtype=float32)
Tensor("batch_normalization_1/cond/Merge:0", shape=(?, 6, 6, 64), dtype=float32)
Tensor("dropout_1_1/cond/Merge:0", shape=(?, 6, 6, 64), dtype=float32)
Tensor("conv2d_2/BiasAdd:0", shape=(?, 2, 2, 128), dtype=float32)
Tensor("batch_normalization_2/cond/Merge:0", shape=(?, 2, 2, 128), dtype=float32)
Tensor("dropout_2/cond/Merge:0", shape=(?, 2, 2, 128), dtype=float32)
Tensor("average_pooling2d/AvgPool:0", shape=(?, 1, 1, 128), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 1, 1, 10), dtype=float32)
Tensor("Reshape:0", shape=(?, 10), dtype=float32)


In [7]:
prediction = layer_12

In [8]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=output_tensor, logits=prediction) # tf.reduce_mean(tf.squared_difference(output_tensor, prediction))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(output_tensor, 1)), "float32"))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    loss_optimization = tf.train.AdamOptimizer().minimize(loss)

In [10]:
images_list = []
for root, folder, files in os.walk("/home/grace/Projects/Mnist/Train_Data/"):
    for file in files :
        images_list.append(root + file)
images_list , test = images_list[0:35000] , images_list[35000:]
print("No of Train Images : ", len(images_list))
print("No of Test Images : ", len(test))


No of Train Images :  35000
No of Test Images :  7000


In [11]:
shuffle(test)

In [12]:
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)

In [ ]:
num_epochs = 10
batch = 10
total_images = len(images_list)
test_images = len(test)
for epoch in range(num_epochs):
    shuffle(images_list)
    acc_list = []
    loss_list = []
    test_acc_list = []
    for i in range(0, total_images-batch, batch):
        image_batch = []
        label_batch = []
        for image in images_list[i:i+batch]:
            single_image = cv2.imread(image, 0)/255.0 - 0.5
            single_image = np.reshape(single_image, (28,28,1))            
            image_batch.append(single_image)
            label = int(image.split("/")[-1].split("_")[0])
            label_batch.append(np.eye(10)[label])
        label_batch = np.array(label_batch, dtype=np.float32)
        if epoch == 0:            
            acc, loss_val = sess.run([accuracy, loss], \
                                     feed_dict={input_tensor :image_batch, output_tensor : label_batch, \
                                               bn_tensor : False, dropout_tensor : 0.0 })
            acc_list.append(acc)
            loss_list.append(loss_val)
        else : 
            opt, acc, loss_val = sess.run([loss_optimization, accuracy, loss], \
                                          feed_dict={input_tensor :image_batch, output_tensor : label_batch, \
                                                    bn_tensor : True, dropout_tensor : 0.2 })
            acc_list.append(acc)
            loss_list.append(loss_val)
    
    shuffle(test)
    for j in range(0, test_images-batch, batch):
        test_image_batch = []
        test_label_batch = []
        for test_image in test[j:j+batch]:
            single_test_image = cv2.imread(test_image, 0)/255.0 - 0.5
            single_test_image = np.reshape(single_test_image, (28,28,1))
            test_image_batch.append(single_test_image)
            test_label = int(test_image.split("/")[-1].split("_")[0])
            test_label_batch.append(np.eye(10)[test_label])
        test_label_batch = np.array(test_label_batch, dtype=np.float32)
        
        test_acc = sess.run([accuracy], \
                            feed_dict={input_tensor :test_image_batch, output_tensor : test_label_batch, \
                                      bn_tensor : True, dropout_tensor : 0.0 })
        test_acc_list.append(test_acc)
        
    print("Epoch : " + str(epoch) + " Loss " +  str(round(np.average(loss_list), 3)) + \
          "  Train Accuracy : " + str(round(np.average(acc_list), 3)) + "  Test Accuracy : " + \
          str(round(np.average(test_acc_list), 3)))
    if epoch % 10 == 0:
        saver = tf.train.Saver()
        saver.save(sess,'./MNIST_Model/MNIST_TF_MODEL_'+str(epoch))
        
        

In [ ]:
frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,
    sess.graph_def,
    ["Reshape"])

with open('./MNIST_Model/./MNIST_TF_FrozenModel.pb', 'wb') as f:
    f.write(frozen_graph_def.SerializeToString())